In [2]:
# Restore the models from local backup
!mkdir -p $HOME/.keras/models
! cp /home/jovyan/work/models/* $HOME/.keras/models/

!ls -ltra $HOME/.keras/models



total 809148
drwxr-sr-x 3 jovyan users      4096 Jul  1 20:39 ..
-rw-r--r-- 1 jovyan users  96112376 Jul  1 20:39 inception_v3_weights_tf_dim_ordering_tf_kernels.h5
-rw-r--r-- 1 jovyan users       120 Jul  1 20:39 keras.json
-rw-r--r-- 1 jovyan users  17225924 Jul  1 20:39 mobilenet_1_0_224_tf.h5
-rw-r--r-- 1 jovyan users 102853048 Jul  1 20:39 resnet50_weights_tf_dim_ordering_tf_kernels.h5
-rw-r--r-- 1 jovyan users 553467096 Jul  1 20:39 vgg16_weights_tf_dim_ordering_tf_kernels.h5
drwxr-sr-x 2 jovyan users      4096 Jul  1 20:39 .
-rw-r--r-- 1 jovyan users  58889256 Jul  1 20:39 vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5


In [50]:
# Remove the top layer of the model
base_model=VGG16(weights='imagenet',input_shape=(224,224,3),include_top=False)

58892288/58889256 [==============================] - 631s 11us/step


### Training the model
* Import Libraries
* Load data
* Pre process the data
* Load the weights of the pretrained model
* Fine tune the model
* Validate the model and adjust the 

### Prediction 
* Get the predictions on the new data 


### 1. Import Libraries

In [3]:
# Import Libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

from glob import glob
from sklearn.model_selection import train_test_split
from skimage.transform import resize

from keras.utils import to_categorical
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import VGG16
from keras.applications import vgg16
from keras.models import Sequential
from keras.layers import Dense, InputLayer


Using TensorFlow backend.


### 2. Load the data

In [4]:
seed = 42
rng = np.random.RandomState(seed)

In [5]:
path ="./../../CV101/datasets/emergency_classification/"
data = pd.read_csv((path+"emergency_classification.csv"))

In [6]:
data.head()

image_names  emergency_or_not
0       0.jpg                 1
1       1.jpg                 1
2       2.jpg                 1
3       3.jpg                 1
4       4.jpg                 1

In [7]:
data.shape

(2352, 2)

In [8]:
X=[]
for localimage in data.image_names:
    img = plt.imread(path+"/images/"+localimage)
    img = img.astype(np.float16)
    X.append(img)

In [9]:
X = np.array(X)

In [10]:
X.shape

(2352, 224, 224, 3)

In [11]:
# Load the dependent variable 
y = data.emergency_or_not.values

In [12]:
y=to_categorical(y)

## 3. Pre-process the data
* Pre-process the data to re size all images to same size
* Split the data into validation and train set

In [13]:
X.max() ,X.min()

(255.0, 0.0)

In [28]:
?preprocess_input

In [14]:
X = preprocess_input(X,mode='tf')

In [15]:
X.max() ,X.min()

(1.0, -1.0)

In [36]:
X_train,X_valid, y_train , y_valid = train_test_split(X, y, test_size=0.3, random_state=42)

In [34]:
X_train.shape

(1646, 224, 224, 3)

In [37]:
X_valid.shape

(706, 224, 224, 3)

## 4. Load the weights of the pretrained model


In [17]:
base_model = vgg16.VGG16(weights='imagenet')

In [18]:
base_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [19]:
# Remove the top layer of the model
base_model=VGG16(weights='imagenet',input_shape=(224,224,3),include_top=False)

In [20]:
base_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

## 5. Fine tune the model
* Extract Features
* Flatten the data 
* Rescale Features
* Create NN Model
* Compile the model
* Train and Validate the data


In [26]:
base_model_pred = base_model.predict(X_train)

In [30]:
X_valid.shape

(1646, 2)

In [38]:
base_model_pred_valid = base_model.predict(X_valid)

In [ ]:
base_model_pred.shape

In [22]:
# Convert into 1D Array
base_model_pred = base_model_pred.reshape(1152,7*7*512)

ValueError: cannot reshape array of size 41294848 into shape (1152,25088)

In [23]:
base_model_pred.shape

(1646, 7, 7, 512)

In [24]:
base_model_pred_valid.shape

NameError: name 'base_model_pred_valid' is not defined

In [ ]:
base_model_pred_valid = base_model_pred_valid.reshape(1152,7*7*512)

In [ ]:
base_model_pred_valid.shape

In [ ]:
base_model_pred_valid

###### Normalize the prediction Values

In [ ]:
base_model_pred.min(), base_model_pred.max()

In [ ]:
base_model_pred_valid.max(), base_model_pred_valid.min()

In [ ]:
max_val = base_model_pred.max()

In [ ]:
max_val_valid = base_model_pred_valid.max()

In [ ]:
base_model_pred = base_model_pred / max_val
base_model_pred_valid = base_model_pred_valid / max_val_valid


In [ ]:
# Normalized the ouputs 
base_model_pred.min(), base_model_pred.max()

In [ ]:
# Create the Neural Network
model = Sequential()
model.add(InputLayer((7*7*512, )))
model.add(Dense(units=1024,activation='relu'))
model.add(Dense(units=2,activation='softmax'))


In [ ]:
model.compile(optimizer='sdg',metrics=['accuracy'],loss='categorical_crossentropy')

In [ ]:
model.summary()

## 6. Validate the model and adjust the 
